In [ ]:
import pandas as pd
import os
import glob
import re
import pathlib
import shutil
import random
import cv2

from pathlib import Path
from darkflow.net.build import TFNet
from scipy.io import loadmat

os.getcwd()

In [ ]:
x = loadmat('../data/external/compcars_data/misc/make_model_name.mat')
carmakers = {i:v[0][0]  for i, v in enumerate(x['make_names'])}
carmakers[2]

In [ ]:
train_images = pd.read_csv('../data/external/compcars_data/train_test_split/classification/train.txt', header=None, delimiter=" ")
test_images = pd.read_csv('../data/external/compcars_data/train_test_split/classification/test.txt', header=None, delimiter=" ")
train_images[:10]


In [ ]:
project_dir = "/Users/az01640/Projets/car-detection"
car_images = glob.glob('{}/data/external/compcars_data/image/*/*.jpg'.format(project_dir))
car_images

In [ ]:
regex = re.compile('image(/.*/).*.jpg')
i=0
for old_file in car_images:
    substring_to_delete = regex.findall(old_file)[0]
    integer_model = int(substring_to_delete.replace("/","")) - 1
    new_file = pathlib.Path(str(old_file).replace(str(substring_to_delete), "/" + carmakers[integer_model] + "/"))
    new_file.parent.mkdir(parents=True, exist_ok=True)
    print(i/len(car_images))
    shutil.copy(str(old_file), str(new_file))
    i+=1

In [ ]:
def get_cars_from_images(tfnet, image_file_path):

    imgcv = cv2.imread(image_file_path)
    result = tfnet.return_predict(imgcv)
    result = sorted([d for d in result if d['label'] == 'car'],key = lambda x: x['confidence'], reverse=True)

    if len(result) > 0:
        
        d = result[0]
        topleft = d['topleft']
        bottomright = d['bottomright']

        xDiff = abs(topleft['x'] - bottomright['x'])
        yDiff = abs(topleft['y'] - bottomright['y'])

        topleft['y'] = max(int(topleft['y'] - 0.1 * yDiff), 0)
        topleft['x'] = max(int(topleft['x'] - 0.1 * xDiff), 0)

        bottomright['y'] = max(int(bottomright['y'] + 0.1 * yDiff), 0)
        bottomright['x'] = max(int(bottomright['x'] + 0.1 * xDiff), 0)

        output_filepath = image_file_path.replace("image","image_gray")
        
        Path(output_filepath).parent.mkdir(parents=True, exist_ok=True)
        img_gray = cv2.cvtColor(imgcv, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(filename=output_filepath,
                    img=img_gray[topleft['y']:bottomright['y'], topleft['x']:bottomright['x']])

    return result

class GetCarsFromImages(object):

    def __init__(self, tfnet):
        self.tfnet = tfnet

    def __call__(self, lat_lng_coord):
        get_cars_from_images(self.tfnet, image_file_path)

In [ ]:
project_dir = "/Users/az01640/Projets/car-detection"

input_images = glob.glob('{}/data/external/compcars_data/image/*/*.jpg'.format(project_dir))

yolo_v2_dir = "{}/models/yolo_v2".format(project_dir)
options = {"model": "{}/cfg/yolo.cfg".format(yolo_v2_dir),
           "load": "{}/bin/yolov2.weights".format(yolo_v2_dir),
           "config": "{}/cfg".format(yolo_v2_dir),
           "threshold": 0.1}

tfnet = TFNet(options)

In [ ]:
input_images

In [ ]:
for img in input_images:
    get_cars_from_images(tfnet, img)

In [ ]:
project_dir = "/Users/az01640/Projets/car-detection"
car_images = glob.glob('{}/data/external/compcars_data/image/*/*/*/'.format(project_dir))
[os.rmdir(f) for f in car_images]
car_images = glob.glob('{}/data/external/compcars_data/image/*/*/'.format(project_dir))
[os.rmdir(f) for f in car_images]

In [ ]:
shutil.rmtree('{}/data/external/compcars_data/label'.format(project_dir))

In [ ]:
shutil.rmtree('{}/data/external/compcars_data/part'.format(project_dir))

In [ ]:
shutil.rmtree('{}/data/external/compcars_data/train_test_split/part'.format(project_dir))

In [ ]:
train_images = list(pd.read_csv('{}/data/external/compcars_data/train_test_split/classification/train.txt'.format(project_dir),
                          header=None)[0])
test_images = list(pd.read_csv('{}/data/external/compcars_data/train_test_split/classification/test.txt'.format(project_dir),
                          header=None)[0])

In [ ]:
regex = re.compile('.*/(.*.jpg)')
train_images = [regex.findall(i)[0] for i in train_images]
test_images = [regex.findall(i)[0] for i in test_images]

In [ ]:
car_images = glob.glob('{}/data/external/compcars_data/image_gray/*/*.jpg'.format(project_dir))
regex = re.compile('image_gray/.*/(.*.jpg)')
mode_list = ['train']*6 + ['test']*4
for i in car_images:
    image_name = regex.findall(i)[0]
    if image_name in train_images:
        mode = "train"
    elif image_name in test_images:
        mode = "test"
    else:
        mode = random.choice(mode_list)
    new_file = pathlib.Path(str(i).replace('image', '{}_image'.format(mode)))
    new_file.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(str(i), str(new_file))

In [ ]:
car_images